In [1]:
import sys
sys.executable

'/home/akarshkumar0101/miniconda3/bin/python'

In [6]:
from hflayers import Hopfield

beta = 2.

In [11]:
hopfield = Hopfield(
    scaling=beta,

    # do not project layer input
    state_pattern_as_static=True,
    stored_pattern_as_static=True,
    pattern_projection_as_static=True,

    # do not pre-process layer input
    normalize_stored_pattern=False,
    normalize_stored_pattern_affine=False,
    normalize_state_pattern=False,
    normalize_state_pattern_affine=False,
    normalize_pattern_projection=False,
    normalize_pattern_projection_affine=False,

    # do not post-process layer output
    disable_out_projection=True)


print("Parameters: ", np.sum([p.numel() for p in hopfield.parameters()], dtype=int))

Parameters:  0


In [20]:
Y = torch.randn(2, 100, 512)
R = torch.randn(2, 3, 512)

In [39]:
# tuple of stored_pattern, state_pattern, pattern_projection
Z = hopfield.forward((Y, R, Y))

In [ ]:
Zp = (beta*(R@Y.transpose(-1, -2))).softmax(dim=-1)@Y

In [80]:
net = torch.nn.MultiheadAttention(512, 1, bias=False, batch_first=True)
print("Parameters: ", np.sum([p.numel() for p in net.parameters()], dtype=int))

Parameters:  1048576


In [58]:
512*512*4

1048576

In [61]:
[p.shape for p in net.parameters()]

[torch.Size([1536, 512]), torch.Size([512, 512])]

In [64]:
net

MultiheadAttention(
  (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=False)
)

In [71]:
[p.shape for p in net.out_proj.parameters()]

[torch.Size([512, 512])]

In [91]:
Zattn, _ = net.forward(R, Y, Y)
Zattn.shape

torch.Size([2, 3, 512])

In [122]:
W1, W2 = list(net.parameters())
Wq, Wk, Wv = W1.chunk(3, dim=0)
Wq, Wk, Wv, W2 = Wq.T, Wk.T, Wv.T, W2.T

Q = R@Wq
K = Y@Wk
V = Y@Wv


In [123]:
Zmine = (Q@K.transpose(-1, -2)/np.sqrt(512)).softmax(dim=-1)@V@W2

In [127]:
(Zmine-Zattn).abs().max()

tensor(5.2154e-08, grad_fn=<MaxBackward1>)

In [140]:
hopfield = Hopfield(
    scaling=float(1./np.sqrt(512)),

    # do not project layer input
    state_pattern_as_static=True,
    stored_pattern_as_static=True,
    pattern_projection_as_static=True,

    # do not pre-process layer input
    normalize_stored_pattern=False,
    normalize_stored_pattern_affine=False,
    normalize_state_pattern=False,
    normalize_state_pattern_affine=False,
    normalize_pattern_projection=False,
    normalize_pattern_projection_affine=False,

    # do not post-process layer output
    disable_out_projection=True)

Zhop = hopfield.forward((K, Q, V))@W2

In [142]:
(Zhop-Zmine).abs().max()

tensor(3.7253e-08, grad_fn=<MaxBackward1>)